In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Dropout, Concatenate, Lambda
from keras.models import Model
import keras.backend as K

# Load the dataset from CSV file
data = pd.read_excel("C:/Users/LENOVO/Desktop/Main Project/dataset/default of credit card clients.xls")



In [2]:
#DRop a column
data=data.drop('Unnamed: 0',axis=1)
data.drop(0, inplace=True)
data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [3]:
# Step 1: Convert non-numeric columns to numeric format
data = data.apply(pd.to_numeric, errors='coerce')

In [4]:

# Separate features and target variable
X = data.drop(columns=['Y']).values
y = data['Y'].values


In [5]:

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:

# Normalize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [25]:
# Reshape input data for LSTM
timesteps = 1  # You can adjust this value based on your data
X_train_lstm = X_train_scaled.reshape(-1, timesteps, X_train_scaled.shape[1])
X_test_lstm = X_test_scaled.reshape(-1, timesteps, X_test_scaled.shape[1])

# Define input layers
input_shape = X_train_lstm.shape[1:]
input_layer = Input(shape=input_shape)

# Embedding layer (optional, you can use it for categorical features)
# embedded = Embedding(input_dim=num_categories, output_dim=embedding_dim)(input_layer)


In [26]:

# Bidirectional LSTM layer
lstm_units = 64
lstm_layer = Bidirectional(LSTM(units=lstm_units, return_sequences=True))(input_layer)


In [27]:
from keras.layers import Attention
# Attention layer using Keras built-in Attention mechanism
attention_weights = Dense(1, activation='softmax')(lstm_layer)
attention_output = K.sum(attention_weights * lstm_layer, axis=1)

# Fully connected layers
dense_units = 32
dense_layer = Dense(units=dense_units, activation='relu')(attention_output)
output_layer = Dense(units=1, activation='sigmoid')(dense_layer)

In [ ]:


# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 32
model.fit(X_train_lstm, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_lstm, y_test))

# Evaluate the model
y_pred = model.predict(X_test_lstm)
y_pred_binary = (y_pred > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred_binary)
print("Test Accuracy:", accuracy)








Epoch 1/10


In [ ]:

# Concatenate LSTM output and attention output
concatenated = Concatenate(axis=-1)([lstm_layer, attention_layer])



In [ ]:
# Fully connected layers
dense_units = 32
dense_layer = Dense(units=dense_units, activation='relu')(concatenated)
output_layer = Dense(units=1, activation='sigmoid')(dense_layer)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)


In [ ]:

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 32
model.fit(X_train_lstm, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_lstm, y_test))


In [ ]:

# Evaluate the model
y_pred = model.predict(X_test_scaled)
y_pred_binary = (y_pred > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred_binary)
print("Test Accuracy:", accuracy)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Dropout, Concatenate, Lambda
from keras.models import Model
import keras.backend as K

# Load the dataset from CSV file
dataset_path = 'credit_card_default.csv'
data = pd.read_csv(dataset_path)

# Separate features and target variable
X = data.drop(columns=['default.payment.next.month']).values
y = data['default.payment.next.month'].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define input layers
input_shape = X_train_scaled.shape[1]
input_layer = Input(shape=(input_shape,))

# Embedding layer (optional, you can use it for categorical features)
# embedded = Embedding(input_dim=num_categories, output_dim=embedding_dim)(input_layer)

# Bidirectional LSTM layer
lstm_units = 64
lstm_layer = Bidirectional(LSTM(units=lstm_units, return_sequences=True))(input_layer)

# Attention layer
def attention_function(inputs):
    hidden_states = inputs
    hidden_size = int(hidden_states.shape[2])
    score = Dense(hidden_size, activation='tanh')(hidden_states)
    attention_weights = Dense(1, activation='softmax')(score)
    context_vector = K.sum(attention_weights * hidden_states, axis=1)
    return context_vector

attention_layer = Lambda(attention_function)(lstm_layer)

# Concatenate LSTM output and attention output
concatenated = Concatenate(axis=-1)([lstm_layer, attention_layer])

# Fully connected layers
dense_units = 32
dense_layer = Dense(units=dense_units, activation='relu')(concatenated)
output_layer = Dense(units=1, activation='sigmoid')(dense_layer)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 32
model.fit(X_train_scaled, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_scaled, y_test))

# Evaluate the model
y_pred = model.predict(X_test_scaled)
y_pred_binary = (y_pred > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred_binary)
print("Test Accuracy:", accuracy)
